<a href="https://colab.research.google.com/github/devangi2000/Deep-Learning/blob/master/resnet50_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import torchvision.transforms as transforms
import torchvision
import torch.nn.functional as F
import pdb


In [2]:
class baseBlock(torch.nn.Module):
  expansion=1
  def __init__(self, input_planes, planes, stride=1, dim_change=None):
    super(baseBlock, self).__init__()
    # declare convolutional layers with batch norm
    self.conv1 = torch.nn.Conv2d(input_planes, planes, stride=stride, kernel_size=3, padding=1) #as described by paper
    self.bn1 = torch.nn.BatchNorm2d(planes)
    self.conv2 = torch.nn.Conv2d(planes, planes, stride=1, kernel_size=3, padding=1)
    self.bn2 = torch.nn.BatchNorm2d(planes)
    self.dim_change = dim_change

  def forward(self, x):
    # save the residue
    res = x
    output = F.relu(self.bn1(self.conv1(x)))
    output = self.bn2(self.conv2(output))

    if self.dim_change is not None:
      res = self.dim_change(res)

    output += res
    output = F.relu(output)

    return output

In [3]:
class bottleNeck(torch.nn.Module):
  expansion = 4
  def __init__(self, input_planes, planes, stride=1, dim_change=None):
    super(bottleNeck, self).__init__()
    # declare convolutional layers with batch norm
    #first layer changes the channels
    self.conv1 = torch.nn.Conv2d(input_planes, planes, kernel_size=1, stride=1)
    self.bn1 = torch.nn.BatchNorm2d(planes)
    # channels remain constant in the second layer; height and width will change because filter size is not 1
    self.conv2 = torch.nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding =1)
    self.bn2 = torch.nn.BatchNorm2d(planes)
    self.conv3 = torch.nn.Conv2d(planes, planes*self.expansion, kernel_size=1)
    self.bn3 = torch.nn.BatchNorm2d(planes*self.expansion)
    
    self.dim_change = dim_change

  def forward(self, x):
    # save the residue
    res = x
    output = F.relu(self.bn1(self.conv1(x)))
    output = F.relu(self.bn2(self.conv2(output)))
    output = self.bn3(self.conv3(output))

    if self.dim_change is not None:
      res = self.dim_change(res)

    output += res
    output = F.relu(output)

    return output

In [4]:
class ResNet(torch.nn.Module):
  def __init__(self, block, num_layers, classes=10):
    super(ResNet, self).__init__()
    # 7x7 convolution of 64 layers; that means 3 channels to 64 channels
    self.input_planes = 64
    self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
    self.bn1 = torch.nn.BatchNorm2d(64) # batch normalize to 64 layers
    #num_layers is a list
    # Layer 1 has 64 channels
    # num_layers[0] tells how many convolutions you want to have
    self.layer1 = self._layer(block, 64, num_layers[0], stride=1) #this returns an entire block
    self.layer2 = self._layer(block, 128, num_layers[1], stride=2)
    self.layer3 = self._layer(block, 256, num_layers[2], stride=2)
    self.layer4 = self._layer(block, 512, num_layers[3], stride=2)
    
    self.averagePool = torch.nn.AvgPool2d(kernel_size=4, stride=1)

    self.fc = torch.nn.Linear(512*block.expansion, classes)

  def _layer(self, block, planes, num_layers, stride=1):
    dim_change = None
    if stride!=1 or planes != self.input_planes*block.expansion:
      dim_change = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes, planes*block.expansion, kernel_size=1, stride=stride),
                                       torch.nn.BatchNorm2d(planes*block.expansion))
    netLayers = [] 
    netLayers.append(block(self.input_planes, planes, stride=stride, dim_change=dim_change))
    self.input_planes = planes * block.expansion

    for i in range(1, num_layers):
      netLayers.append(block(self.input_planes, planes))
      self.input_planes = planes * block.expansion

    return torch.nn.Sequential(*netLayers)

  def forward(self,x):
    x = F.relu(self.bn1(self.conv1(x)))

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = F.avg_pool2d(x, 4)
    x = x.view(x.size(0), -1)
    x = self.fc(x)

    return x




In [5]:
def test():

  # convert data from PIL to Tensor
  transform = transforms.Compose(
      [transforms.ToTensor(),
       transforms.Normalize((0.5, 0.5, 0.5), (0.5,0.5,0.5))]
  )
   # Load train and test set
  train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform = transform)
  trainset = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True)

  test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
  testset = torch.utils.data.DataLoader(test, batch_size=128, shuffle=False)
  device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  print(device)

   # Resnet-18
   # net = ResNe(baseBlock, [2, 2, 2, 2], 10)
    #Resnet-50
  
  net = ResNet(bottleNeck, [3,4,6,3])
  net.to(device)
  costFunc = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(net.parameters(), lr=0.02, momentum=0.9)

  for epoch in range(100):
    closs = 0
    for i, batch in enumerate(trainset, 0):
      data, output = batch
      data, output = data.to(device), output.to(device)
      prediction = net(data)
      loss = costFunc(prediction, output)
      closs = loss.item()

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if i%100 == 0:
        print('[%d %d] loss : %.4f'% (epoch+1, i+1, closs/1000))

    correctHits = 0
    total = 0
    for batches in testset:
      data, output = batches
      data, output = data.to(device), output.to(device)
      prediction = net(data)
      _, prediction = torch.max(prediction.data, 1) #returns max as well as its index
      total += output.size(0)
      correctHits += (prediction==output).sum().item()
    print('Accuracy on epoch ', epoch+1, ' = ', str((correctHits/total)*100))

  correctHits = 0
  total = 0
  for batches in testset:
    data, output = batches
    data, output = data.to(device), output.to(device)
    prediction = net(data)
    _, prediction = torch.max(prediction.data, 1) #returns max as well as its index
    total += output.size(0)
    correctHits += (prediction==output).sum().item()
  print('Accuracy on epoch ', epoch+1, ' = ', str((correctHits/total)*100))

In [ ]:
if __name__=='__main__':
  test()

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
cuda:0
[1 1] loss : 0.0026
[1 101] loss : 0.0024
[1 201] loss : 0.0017
[1 301] loss : 0.0017
Accuracy on epoch  1  =  45.32
[2 1] loss : 0.0016
[2 101] loss : 0.0015
[2 201] loss : 0.0014
[2 301] loss : 0.0014
Accuracy on epoch  2  =  54.66
[3 1] loss : 0.0012
[3 101] loss : 0.0011
[3 201] loss : 0.0014
[3 301] loss : 0.0009
Accuracy on epoch  3  =  64.68
[4 1] loss : 0.0009
[4 101] loss : 0.0010
[4 201] loss : 0.0010
[4 301] loss : 0.0009
Accuracy on epoch  4  =  68.89
[5 1] loss : 0.0006
[5 101] loss : 0.0008
[5 201] loss : 0.0008
[5 301] loss : 0.0007
Accuracy on epoch  5  =  72.07000000000001
[6 1] loss : 0.0006
[6 101] loss : 0.0007
[6 201] loss : 0.0006
[6 301] loss : 0.0006
Accuracy on epoch  6  =  73.26
[7 1] loss : 0.0006
[7 101] loss : 0.0005
[7 201] loss : 0.0006
[7 301] loss : 0.0006
Accuracy on epoch  7  =  75.08
[8 1] loss : 0.0004
[8 101] loss : 0.0003
[8 201] loss : 0.0004
[8 301] l